In [38]:
from numpy import *
from matplotlib import style
style.use("ggplot")
from sklearn import svm
from scipy import sparse as sp
from sklearn.decomposition import TruncatedSVD
from scipy import sparse as sp
from sklearn import decomposition

import matplotlib
import csv
from scipy import sparse
%pylab
%matplotlib inline
from matplotlib import pyplot as plt

from sklearn import cluster


Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [30]:
description = np.genfromtxt('description.csv',delimiter=',')

In [10]:
known = np.genfromtxt('category_partial_supervision.csv',delimiter=',')
known = known[1:]

In [12]:
social = np.genfromtxt('social_and_evolution.csv',delimiter=',')

In [31]:
def toDense(data,m,n):
    X_dat=data
    X_c=sp.coo_matrix((m,n))
    X_c.data=X_dat[:,2].astype(float64)
    X_c.row=X_dat[:,0].astype(int32)
    X_c.col=X_dat[:,1].astype(int32)
    X=X_c.todense()
    return X

In [34]:
description= toDense(description,1829,8000)
pca = decomposition.PCA(n_components=6000)
pca.fit(description)
description = pca.transform(description)

In [39]:
def cca(XA,XB,d):
    XA_c=matrix(XA-average(XA,0))
    XB_c=matrix(XB-average(XB,0))
    sig11=transpose(XA_c)*XA_c
    sig12=transpose(XA_c)*XB_c
    sig22=transpose(XB_c)*XB_c
    sig21=transpose(XB_c)*XA_c
    lambdaA,wA=linalg.eigh(inv(sig11)*sig12*inv(sig22)*sig21)
    ind=argsort(lambdaA)[::-1]
    return XA_c*matrix(wA[:,ind[0:d]])

In [40]:
cca1 = cca(description,social,2)

In [41]:
cca1

matrix([[-0.03266063, -0.000398  ],
        [ 0.06955014,  0.01641089],
        [ 0.01900177,  0.03033583],
        ..., 
        [-0.01492065,  0.01251353],
        [-0.04638806, -0.01961247],
        [ 0.05049594,  0.00500898]])

In [44]:
clf = svm.SVC(kernel='rbf', C = 1.0)

In [88]:
known_cca1 = zeros([shape(known)[0],2])

In [89]:
known_cca1 = zeros([shape(known)[0],2])
k = 0
for i in known[:,0]:
    known_cca1[k,0]=cca1[i,0]
    known_cca1[k,1]=cca1[i,1]
    k+=1

In [90]:
known_cca1

array([[  4.60360950e-03,   3.64252164e-03],
       [ -6.78939780e-02,  -2.31238332e-02],
       [  3.10874892e-03,  -8.22118017e-05],
       [  2.05275751e-02,  -3.06052126e-03],
       [ -2.44768421e-02,  -2.17024802e-02],
       [  1.07583737e-01,   1.04030123e-02],
       [ -4.42294585e-02,   8.58090763e-03],
       [  3.27400901e-02,  -8.83922638e-03],
       [ -1.39877082e-02,  -1.91581467e-02],
       [  4.59582480e-03,  -1.61309814e-02],
       [ -1.27839346e-03,  -1.52433733e-02],
       [  6.36234059e-02,  -2.56639560e-02],
       [ -1.47771019e-02,  -5.67257051e-02],
       [  1.20258714e-03,  -2.42288835e-02],
       [  2.05509036e-02,   3.53310271e-02],
       [  3.65724367e-02,  -6.37542199e-02],
       [ -2.19331655e-02,   1.91183030e-02],
       [ -2.38081640e-02,   3.94119495e-02],
       [ -5.53272330e-02,  -2.58622251e-02],
       [  3.46272060e-02,   2.07013758e-02],
       [  2.91376694e-02,  -7.21783925e-03],
       [  3.68610647e-04,   1.27384109e-03],
       [ -

In [91]:
clf.fit(known_cca1,known[:,1])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [108]:
known_cca1

array([[  4.60360950e-03,   3.64252164e-03],
       [ -6.78939780e-02,  -2.31238332e-02],
       [  3.10874892e-03,  -8.22118017e-05],
       [  2.05275751e-02,  -3.06052126e-03],
       [ -2.44768421e-02,  -2.17024802e-02],
       [  1.07583737e-01,   1.04030123e-02],
       [ -4.42294585e-02,   8.58090763e-03],
       [  3.27400901e-02,  -8.83922638e-03],
       [ -1.39877082e-02,  -1.91581467e-02],
       [  4.59582480e-03,  -1.61309814e-02],
       [ -1.27839346e-03,  -1.52433733e-02],
       [  6.36234059e-02,  -2.56639560e-02],
       [ -1.47771019e-02,  -5.67257051e-02],
       [  1.20258714e-03,  -2.42288835e-02],
       [  2.05509036e-02,   3.53310271e-02],
       [  3.65724367e-02,  -6.37542199e-02],
       [ -2.19331655e-02,   1.91183030e-02],
       [ -2.38081640e-02,   3.94119495e-02],
       [ -5.53272330e-02,  -2.58622251e-02],
       [  3.46272060e-02,   2.07013758e-02],
       [  2.91376694e-02,  -7.21783925e-03],
       [  3.68610647e-04,   1.27384109e-03],
       [ -

In [104]:
for i in known:
    print str(i[0]) + " " + str(i[1]) + " " + str(int(clf.predict(cca1[i[0]])))

80.0 0.0 7
452.0 0.0 0
1688.0 0.0 7
262.0 1.0 11
449.0 1.0 9
1714.0 1.0 1
396.0 2.0 0
1032.0 2.0 3
1261.0 2.0 9
226.0 3.0 7
1189.0 3.0 7
1200.0 3.0 3
501.0 4.0 10
1361.0 4.0 4
1740.0 4.0 11
810.0 5.0 10
1412.0 5.0 12
1173.0 5.0 8
1553.0 6.0 0
1576.0 6.0 11
1650.0 6.0 3
270.0 7.0 7
1798.0 7.0 12
1800.0 7.0 6
1385.0 8.0 8
1453.0 8.0 12
1686.0 8.0 9
598.0 9.0 10
670.0 9.0 7
684.0 9.0 0
900.0 10.0 3
926.0 10.0 9
1191.0 10.0 3
232.0 11.0 11
643.0 11.0 7
1539.0 11.0 11
491.0 12.0 4
594.0 12.0 8
1168.0 12.0 11
